# Association Analysis

Note:
 * You will need the package `mlxtend`.

The following code obtain the data from the UCI data repository, and to read it into a dataframe.

In [1]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

import requests, os
xlUrl = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx'
xlFile = 'data/Online Retail.xlsx'
dataFile = xlFile
url = xlUrl
if not os.path.exists('data'):
    os.makedirs('data')
if not os.path.isfile(dataFile):
    r = requests.get(url)
    with open(dataFile, 'wb') as f:
        f.write(r.content)
if (dataFile == xlFile):
    df = pd.read_excel(dataFile)
else:
    df = pd.read_csv(dataFile)
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


The following lines tidy up the description column, ensure that every row is assigned an invoice number, and that they represent actual transactions.

In [2]:
df['Description'] = df['Description'].str.strip()
df.dropna(axis=0, subset=['InvoiceNo'], inplace=True)
df['InvoiceNo'] = df['InvoiceNo'].astype('str')
df = df[~df['InvoiceNo'].str.contains('C')]
df.shape

(532621, 8)

In [3]:
# Get the unique list of countries
print(df['Country'].unique())

['United Kingdom' 'France' 'Australia' 'Netherlands' 'Germany' 'Norway'
 'EIRE' 'Switzerland' 'Spain' 'Poland' 'Portugal' 'Italy' 'Belgium'
 'Lithuania' 'Japan' 'Iceland' 'Channel Islands' 'Denmark' 'Cyprus'
 'Sweden' 'Finland' 'Austria' 'Bahrain' 'Israel' 'Greece' 'Hong Kong'
 'Singapore' 'Lebanon' 'United Arab Emirates' 'Saudi Arabia'
 'Czech Republic' 'Canada' 'Unspecified' 'Brazil' 'USA'
 'European Community' 'Malta' 'RSA']


I'm selecting data from germany and performing analysis on that as it has sufficient data.

From the below output it can be observered that Germany has 9042 records. 

In [5]:
df['Country'].value_counts()

United Kingdom          487622
Germany                   9042
France                    8408
EIRE                      7894
Spain                     2485
Netherlands               2363
Belgium                   2031
Switzerland               1967
Portugal                  1501
Australia                 1185
Norway                    1072
Italy                      758
Channel Islands            748
Finland                    685
Cyprus                     614
Sweden                     451
Unspecified                446
Austria                    398
Denmark                    380
Poland                     330
Japan                      321
Israel                     295
Hong Kong                  284
Singapore                  222
Iceland                    182
USA                        179
Canada                     151
Greece                     145
Malta                      112
United Arab Emirates        68
European Community          60
RSA                         58
Lebanon 

In [6]:
df.isna().sum()

InvoiceNo           0
StockCode           0
Description      1455
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     134697
Country             0
dtype: int64

In [7]:
country_transactions = df[df['Country'] == 'Germany'].reset_index()

In [8]:
len(country_transactions)

9042

In [9]:
len(country_transactions['Description'].unique())

1695

In [10]:
len(country_transactions['InvoiceNo'].unique())

457

Dropping columns which are not needed before pivoting the table

In [11]:
country_transactions
reduced_transactions = country_transactions.drop(['StockCode', 'InvoiceDate','UnitPrice','CustomerID','Country'],axis=1) 

In [12]:
reduced_transactions

,index,InvoiceNo,Description,Quantity
0,1109,536527,SET OF 6 T-LIGHTS SANTA,6
1,1110,536527,ROTATING SILVER ANGELS T-LIGHT HLDR,6
2,1111,536527,MULTI COLOUR SILVER T-LIGHT HOLDER,12
3,1112,536527,5 HOOK HANGER MAGIC TOADSTOOL,12
4,1113,536527,3 HOOK HANGER MAGIC GARDEN,12
...,...,...,...,...
9037,541801,581578,SET OF 4 PANTRY JELLY MOULDS,12
9038,541802,581578,PACK OF 20 NAPKINS PANTRY DESIGN,12
9039,541803,581578,PACK OF 20 NAPKINS RED APPLES,12
9040,541804,581578,JINGLE BELL HEART ANTIQUE SILVER,12


In [13]:
len(reduced_transactions['InvoiceNo'].unique())

457

### Using pandas pivot tables.

In [14]:
pivot_table = pd.pivot_table(reduced_transactions, values='Quantity',index='InvoiceNo', columns='Description')

In [15]:
pivot_table.drop(['POSTAGE'], axis=1, inplace=True)
pivot_table

Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE SKULLS,...,YULETIDE IMAGES GIFT WRAP SET,ZINC HEART T-LIGHT HOLDER,ZINC STAR T-LIGHT HOLDER,ZINC BOX SIGN HOME,ZINC FOLKART SLEIGH BELLS,ZINC HEART LATTICE T-LIGHT HOLDER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC WILLIE WINKIE CANDLE STICK
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536527,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
536840,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
536861,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
536967,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
536983,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581266,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
581494,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
581570,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Filling all NaN values with 0

In [16]:
baskets = pivot_table.fillna(0).astype('int')

In [17]:
baskets

Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE SKULLS,...,YULETIDE IMAGES GIFT WRAP SET,ZINC HEART T-LIGHT HOLDER,ZINC STAR T-LIGHT HOLDER,ZINC BOX SIGN HOME,ZINC FOLKART SLEIGH BELLS,ZINC HEART LATTICE T-LIGHT HOLDER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC WILLIE WINKIE CANDLE STICK
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536527,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536840,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536861,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536967,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536983,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581266,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
581494,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
581570,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


The below code will replace all cells with positive integer with True and other's with False. We do this because it is computationally faster to work on boolean data than numberic data and also avoid the warning raised when finding frequent items

In [18]:
baskets = (baskets > 0)

In [19]:
frequent_itemsets = apriori(baskets, min_support=0.02,use_colnames=True)

In [20]:
len(frequent_itemsets)

528

Using mlxtend's `association_rules` function to find the association rules where the minimum lift threshold is 1.
Sort them in non-increasing order of lift (largest to smallest).

In [23]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1).sort_values(by=['lift'], ascending=False)

In [24]:
len(rules)

738

In [26]:
rules.sort_values(by='lift',ascending=False)
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
126,(DOLLY GIRL CHILDRENS BOWL),(DOLLY GIRL CHILDRENS CUP),0.026258,0.024070,0.024070,0.916667,38.083333,0.023438,11.711160,1.000000
127,(DOLLY GIRL CHILDRENS CUP),(DOLLY GIRL CHILDRENS BOWL),0.024070,0.026258,0.024070,1.000000,38.083333,0.023438,inf,0.997758
68,(BLUE VINTAGE SPOT BEAKER),(PINK VINTAGE SPOT BEAKER),0.030635,0.024070,0.024070,0.785714,32.642857,0.023333,4.554340,1.000000
69,(PINK VINTAGE SPOT BEAKER),(BLUE VINTAGE SPOT BEAKER),0.024070,0.030635,0.024070,1.000000,32.642857,0.023333,inf,0.993274
44,(BLUE OWL SOFT TOY),(PINK OWL SOFT TOY),0.032823,0.028446,0.021882,0.666667,23.435897,0.020948,2.914661,0.989819


Add new column storing the rule length.

In [27]:
rules["rule_len"] = rules.apply(lambda row: len(row["antecedents"])+len(row["consequents"]), axis=1)
rules[rules["rule_len"]==2].sort_values(by='lift',ascending=False).head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric,rule_len
126,(DOLLY GIRL CHILDRENS BOWL),(DOLLY GIRL CHILDRENS CUP),0.026258,0.024070,0.024070,0.916667,38.083333,0.023438,11.711160,1.000000,2
127,(DOLLY GIRL CHILDRENS CUP),(DOLLY GIRL CHILDRENS BOWL),0.024070,0.026258,0.024070,1.000000,38.083333,0.023438,inf,0.997758,2
68,(BLUE VINTAGE SPOT BEAKER),(PINK VINTAGE SPOT BEAKER),0.030635,0.024070,0.024070,0.785714,32.642857,0.023333,4.554340,1.000000,2
69,(PINK VINTAGE SPOT BEAKER),(BLUE VINTAGE SPOT BEAKER),0.024070,0.030635,0.024070,1.000000,32.642857,0.023333,inf,0.993274,2
44,(BLUE OWL SOFT TOY),(PINK OWL SOFT TOY),0.032823,0.028446,0.021882,0.666667,23.435897,0.020948,2.914661,0.989819,2


**Sample use case**

 * Record at index 68 says transactions that contain (BLUE VINTAGE SPOT BEAKER) are more likely to contain (PINK VINTAGE SPOT BEAKER)
 * Record at index 69 says transactions that contain (PINK VINTAGE SPOT BEAKER) are more likey to contain (BLUE VINTAGE SPOT BEAKER)
 * Since both records have the same lift value we can look into some other metrics to determine what to suggest.
 * The confidence level at index 68 is 1. Although this could mean high association, it could also mean that it is a mandate i.e., Customer who buy (PINK VINTAGE SPOT BEAKER) are required to buy (BLUE VINTAGE SPOT BEAKER) and that does create much business value.
 * Index 69 on the other hand has a confidence value of 0.785714 which means that customers who buy (BLUE VINTAGE SPOT BEAKER) are 78% more likely to buy (PINK VINTAGE SPOT BEAKER).
 * In conclusion it is better to suggest 'PINK VINTAGE SPOT BEAKER' to someone who already had 'BLUE VINTAGE SPOT BEAKER' and not vice-versa.